In [1]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
import train
import matplotlib.gridspec as grd

# sns.set(style="black")
# plt.style.use('dark_background')
outcome_def = 'iai_intervention' # output
%matplotlib inline
%load_ext autoreload
%autoreload 2

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.

# combined

In [2]:
dummy = True
df_pecarn = data_pecarn.get_data(dummy=dummy)
feat_names, pecarn_feat_names = data.get_feat_names(df_pecarn)

df_psrc = data_psrc.get_data(dummy=dummy)
feat_names, psrc_feat_names = data.get_feat_names(df_psrc)

computing pecarn preprocessing...
computing psrc preprocessing...


In [11]:
feat_names = [k for k in psrc_feat_names if k in pecarn_feat_names]
meta = [outcome_def, 'cv_fold', 'dset']
common_feats = feat_names + meta
df1 = df_pecarn[common_feats]
df2 = df_psrc[common_feats]
df = pd.concat((df1, df2), axis=0)

# don't include doubles for binary features
feat_names = [f for f in feat_names if not f.endswith('_no')]
# print(len(feat_names), feat_names)

# don't include race or 
df['AbdTenderDegree_Mild'] = df['AbdTenderDegree_unknown'] | df['AbdTenderDegree_Mild']
feat_names = [f for f in feat_names if not 'Race' in f
              and '_or_' not in f
              and not f == 'AbdTenderDegree_unknown']
print(len(feat_names), feat_names)

27 ['CostalTender', 'VomitWretch_yes', 'RecodedMOI_Fall from an elevation', 'AbdTenderDegree_Mild', 'InitSysBPRange', 'RecodedMOI_unknown', 'AbdTenderDegree_Moderate', 'AbdDistention_unknown', 'AbdDistention_yes', 'AbdomenPain_yes', 'RecodedMOI_Bike collision/fall', 'RecodedMOI_Pedestrian/bicyclist struck by moving vehicle', 'RecodedMOI_Motorcycle/ATV/Scooter collision', 'SeatBeltSign_yes', 'InitHeartRate', 'Hypotension', 'RtCostalTender', 'VomitWretch_unknown', 'RecodedMOI_Motor vehicle collision', 'RecodedMOI_Object struck abdomen', 'AbdTenderDegree_Severe', 'Age', 'LtCostalTender', 'GCSScore', 'AbdTrauma_yes', 'ThoracicTrauma_yes', 'DecrBreathSound_yes']


In [12]:
train_idxs = data.pecarn_train_idxs
test_idxs = data.psrc_train_idxs

# predict

In [ ]:
for balancing in ['ros', 'smote']:
    for balancing_ratio in [100, 5]: 
        for model_type in ['logistic', 'dt']: #, 'rf', 'mlp2', 'svm']): # 'rf', 'mlp2', 'svm', 'gb'
            for feature_selection in ['select_lasso', 'select_rf']: # select_lasso, select_rf, None
                for feature_selection_num in tqdm([5, 10, len(feat_names)]): # 34 total feats
#                 feats = feat_names[:num_feats]
                    out_dir = f'results/jun3_2'
                    os.makedirs(out_dir, exist_ok=True)
#                     f'{model_type}_{num_feats}_{balancing}={balancing_ratio}
                    out_name = f'{model_type}_{feature_selection}={feature_selection_num}_{balancing}={balancing_ratio}'
                    train.train(df,
                                feat_names=feat_names,
                                model_type=model_type, 
                                balancing=balancing,
                                outcome_def=outcome_def,
                                balancing_ratio=balancing_ratio,
                                out_name=f'{out_dir}/{out_name}.pkl',
                                feature_selection=feature_selection,
                                feature_selection_num=feature_selection_num,
                                train_idxs=train_idxs,
                                test_idxs=test_idxs)


  0%|          | 0/3 [00:00<?, ?it/s]/accounts/projects/vision/.local/lib/python3.7/site-packages/imblearn/utils/_validation.py:253: UserWarning: After over-sampling, the number of samples (787100) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7871)
  n_samples_majority))
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/imblearn/utils/_validation.py:253: UserWarning: After over-sampling, the number of samples (791500) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7915)
  n_samples_majority))
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing i